In [1]:
from lda import lda
from lda import load_docs_in_bow, process_vectors, average_document_length, assign_topic, get_word_assignment_info, get_document_assignment_info, get_term_probability, compute_cf, query_score
import numpy as np
from collections import Counter

In [2]:
N_TOPIC = 10
EPOCHS = 1000
ALPHA = 0.5
BETA = 0.01
BATCH = 50

In [3]:
path_to_doc = "../data/docs_trec_covid"

vocabulary, vectors, doc_id = load_docs_in_bow(path_to_doc, n_doc=10)

dict_document = process_vectors(vectors)

In [4]:
print(len(vocabulary))
print(average_document_length(dict_document))

551
73.1


In [5]:
def int2string(dict_document, doc_id, vocabulary):
    new_dict_document = {}
    
    for document, word_topic in dict_document.items():
        new_dict_document[doc_id[document]] = list(map(lambda x: (vocabulary[x[0]], x[1]), word_topic))
        
    return new_dict_document

In [6]:
topic_assignment = assign_topic(dict_document, N_TOPIC)

In [7]:
for i in range(EPOCHS // BATCH):
    topic_assignment, is_converge = lda(topic_assignment, N_TOPIC, len(vocabulary), BATCH, ALPHA, BETA)
    print(f'after {(i + 1) * BATCH} iterations, learning converges : {is_converge}\n')
    if is_converge:
        break

------- Starting 1 iteration -------
------- Starting 2 iteration -------
------- Starting 3 iteration -------
------- Starting 4 iteration -------
------- Starting 5 iteration -------
------- Starting 6 iteration -------
------- Starting 7 iteration -------
------- Starting 8 iteration -------
------- Starting 9 iteration -------
------- Starting 10 iteration -------
------- Starting 11 iteration -------
------- Starting 12 iteration -------
------- Starting 13 iteration -------
------- Starting 14 iteration -------
------- Starting 15 iteration -------
------- Starting 16 iteration -------
------- Starting 17 iteration -------
------- Starting 18 iteration -------
------- Starting 19 iteration -------
------- Starting 20 iteration -------
------- Starting 21 iteration -------
------- Starting 22 iteration -------
------- Starting 23 iteration -------
------- Starting 24 iteration -------
------- Starting 25 iteration -------
------- Starting 26 iteration -------
------- Starting 27 i

In [8]:
print(is_converge)

True


In [9]:
# def get_word_wise_assignment(topic_assignment, vocabulary, n_topics, n_vocabularies):
#     matrix_word_topic = np.zeros((n_vocabularies, n_topics))
#     for word_topic in topic_assignment.values():
#         for word, topic in word_topic:
#             matrix_word_topic[word, topic] += 1
# 
#     return matrix_word_topic
# 
# 
# def get_word_assignment_info(topic_assignment, vocabulary, n_topics, n_vocabularies, top_topic=5, top_word=10):
#     word_topic_matrix = get_word_wise_assignment(topic_assignment, vocabulary, n_topics, n_vocabularies)
#     top_topic_index = np.argsort(word_topic_matrix.sum(axis=0))[::-1]
#     topic_percentage = np.sort(word_topic_matrix.sum(axis=0) / word_topic_matrix.sum())[::-1]
#     print(f'------- The top {top_topic} topics-------\n')
#     print(f'It\'s top {top_word} words:')
#     for t in range(top_topic):
#         print(f'- topic {top_topic_index[t]} ({topic_percentage[t] * 100 :.2f}%)')
#         word_assigment_vector = word_topic_matrix[:, top_topic_index[t]]
#         top_word_index = np.argsort(word_assigment_vector)[::-1]
#         word_percentage = np.sort(word_assigment_vector / word_assigment_vector.sum())[::-1]
#         for w in range(top_word):
#             print(f'\t- {vocabulary[top_word_index[w]]} ({word_percentage[w] * 100:.2f}%)')

In [10]:
get_word_assignment_info(topic_assignment, vocabulary, N_TOPIC, len(vocabulary))

------- The top 5 topics -------

It's top 10 words:
- topic 0 (13.00%)
	- lung (3.16%)
	- mani (3.16%)
	- disea (3.16%)
	- protein (3.16%)
	- review (3.16%)
	- evid (3.16%)
	- oxid (3.16%)
	- mediat (3.16%)
	- inflammatori (3.16%)
	- pulmonari (3.16%)
- topic 2 (11.63%)
	- genom (2.35%)
	- respon (2.35%)
	- express (2.35%)
	- alter (2.35%)
	- respiratori (2.35%)
	- epith (2.35%)
	- isol (2.35%)
	- specif (2.35%)
	- mice (2.35%)
	- various (2.35%)
- topic 7 (9.99%)
	- patient (2.74%)
	- common (2.74%)
	- abdulaziz (1.37%)
	- restrict (1.37%)
	- proven (1.37%)
	- publish (1.37%)
	- quarter (1.37%)
	- question (1.37%)
	- random (1.37%)
	- rate (1.37%)
- topic 8 (9.85%)
	- respond (4.17%)
	- addit (4.17%)
	- import (4.17%)
	- influenc (2.78%)
	- includ (2.78%)
	- respect (2.78%)
	- human (2.78%)
	- reaction (2.78%)
	- apoptosi (2.78%)
	- discuss (2.78%)
- topic 9 (9.71%)
	- result (5.63%)
	- respiratori (4.23%)
	- occur (4.23%)
	- recent (4.23%)
	- present (2.82%)
	- high (2.82%)
	- debat

In [11]:
# def get_document_wise_assignment(topic_assignment, n_topics):
#     n_documents = len(topic_assignment)
#     matrix_document_topic = np.zeros((n_documents, n_topics))
#     for document, word_topic in topic_assignment.items():
#         for _, topic in word_topic:
#             matrix_document_topic[document, topic] += 1
# 
#     return matrix_document_topic
# 
# 
# def get_document_assignment_info(topic_assignment, doc_id, n_topics, n_documents=5):
#     document_topic_matrix = get_document_wise_assignment(topic_assignment, n_topics)
#     print('The distribution of topics to documents:')
#     for d in range(n_documents):
#         document_topic_vector = document_topic_matrix[d]
#         top_topic_index = np.argsort(document_topic_vector)[::-1]
#         topic_percentage = np.sort(document_topic_vector / document_topic_vector.sum())[::-1]
#         string_topic = f'{topic_percentage[0]:.3f} * topic{top_topic_index[0]}'
#         
#         for topic, percentage in zip(top_topic_index[1:], topic_percentage[1:]):
#             if percentage <= 0.0001:
#                 break
#             else:
#                 string_topic += ' + '
#                 string_topic += f'{percentage:.3f} * topic{topic}'
#                 
#         print(f'\t- {doc_id[d]} : {string_topic}')

In [12]:
get_document_assignment_info(topic_assignment, doc_id, N_TOPIC)

The distribution of topics to documents:
	- ug7v899j : 0.539 * topic7 + 0.337 * topic1 + 0.101 * topic9 + 0.011 * topic6 + 0.011 * topic2
	- 02tnwd4m : 0.409 * topic0 + 0.364 * topic9 + 0.136 * topic6 + 0.045 * topic5 + 0.015 * topic7 + 0.015 * topic3 + 0.015 * topic2
	- ejv2xln0 : 0.412 * topic2 + 0.299 * topic0 + 0.144 * topic8 + 0.124 * topic6 + 0.021 * topic3
	- 2b73a28n : 0.742 * topic0 + 0.226 * topic3 + 0.032 * topic6
	- 9785vg6d : 0.646 * topic2 + 0.200 * topic1 + 0.138 * topic8 + 0.015 * topic5


In [13]:
print(type(vocabulary))


<class 'numpy.ndarray'>


In [14]:
cf_counter = compute_cf(dict_document)
all_cf = 0
for i in range(len(vocabulary)):
    all_cf += cf_counter[i]

In [19]:
query = ['antigen', 'statist']
int_query = list(map(lambda x: list(vocabulary).index(x), query))

In [21]:
scores = query_score(int_query, dict_document, topic_assignment, N_TOPIC, len(vocabulary), ALPHA, BETA)

In [22]:
print(scores)

[-13.918959621243076, -13.299507714777924, -13.287287685503077, -13.34673788573024, -13.91580678396462, -13.377620822022415, -13.752414647022555, -12.930456239224334, -13.574366427789565, -12.969297017619592]
